In [88]:
from simple_cysh import * # includes pandas as pd

In [73]:
# deermine which fields are actively used in our system within the Assesment__c object
single_assmt = cysh.Assesment__c.get('a041a00000EmURoAAN')
single_assmt = pd.DataFrame(list(single_assmt.items()))
list(single_assmt.loc[~single_assmt[1].isnull()][0])

['attributes',
 'Id',
 'IsDeleted',
 'Name',
 'CreatedDate',
 'CreatedById',
 'LastModifiedDate',
 'LastModifiedById',
 'SystemModstamp',
 'LastViewedDate',
 'LastReferencedDate',
 'Student__c',
 'DCCAS_Math_Goal_Met__c',
 'DCCAS_Reading_Goal_Met__c',
 'Date_Administered__c',
 'Date__c',
 'Goal_Met__c',
 'Location__c',
 'School__c',
 'Show_On_Report_Card__c',
 'Show_On_Transcript__c',
 'Student_Grade__c',
 'Student_Id__c',
 'Student_Name__c',
 'Testing_Grade_Level__c',
 'Type__c',
 'Year_Over_Year__c',
 'X0_to_300_Scaled_Score__c',
 'Student_Last_Name__c',
 'Student_Gender__c',
 'Student_First_Name__c',
 'Student_DOB__c']

In [ ]:
# get all assessments with desired fields
assmt_df = get_cysh_df('Assesment__c', ['Id', 'Type__c', 'Date_Administered__c',
                                        'X0_to_300_Scaled_Score__c', 'Student__c',
                                        'Testing_Grade_Level__c'], rename_id=True)

# get assessment type
assmt_types = get_cysh_df('Picklist_Value__c', ['Id', 'Name'])
assmt_types = assmt_types.rename(columns={'Id': 'Type__c', 'Name':'Assessment Type'})
assmt_df = assmt_df.merge(assmt_types, how='left', on='Type__c')
del assmt_df['Type__c']

# filter for valid assessments
assmt_df = assmt_df.loc[assmt_df['Assessment Type'].str.contains('NWEA')
                        & ~assmt_df['X0_to_300_Scaled_Score__c'].isnull()
                        & (assmt_df['X0_to_300_Scaled_Score__c']>0)]

In [ ]:
# get current year grade level for each student
stu_sec_df = get_cysh_df('Student_Section__c', ['Student__c', 'Student_Grade__c'])
# match in grade level
# for 

In [95]:
# pivot: one student per row, assessments as columns
assmt_df['Key'] = assmt_df['Assessment Type'] + '_' + assmt_df['Date_Administered__c']

assmt_pivot = assmt_df.pivot_table(index=['Student__c', 'Testing_Grade_Level__c'], columns='Key', values='X0_to_300_Scaled_Score__c')
assmt_pivot.head(0)

,Key,NWEA - ELA_2017-07-01,NWEA - ELA_2017-09-20,NWEA - ELA_2018-01-09,NWEA - MATH_2017-07-01,NWEA - MATH_2017-09-20,NWEA - MATH_2018-01-09
Student__c,Testing_Grade_Level__c,,,,,,


# Generate Assessment Targets
* calculate fall to spring targets
* determine student growth
* associate students with ACM